# 01. Web Scrapping : Cars Raw

The goal of this notebook is to obtain the whole data base for the project by applying web scrapping on the website Autoscout24

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt

from time import sleep
from time import time
from random import randint

In [2]:
# Links for the different categories
small_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=1&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
sedan_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=6&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
familiar_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1& cy=E&body=5&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
minivan_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
coupe_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=3&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
cabrio_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=2&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
suv_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=4&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
van_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=13&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"
other_cars_url = "https://www.autoscout24.es/lst/?sort=age&desc=1&bcol= 1 &offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page= 1 &cy=E&body=7&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"



In [3]:
seller = np.array(["D","P"])
seller

array(['D', 'P'], dtype='<U1')

In [4]:
seller_dictionary = {'D': 'Dealer', 
                      'P': 'Private' }

The colour is not available for srapping as we cannot see the field in the main search, so the techical solution is to include it on the website link when setting coding the webpage to be scrapped as if we include &bcol=10& on the path we are extracting cars for colour 10 in this case

#example of the colour code in the link:
"https://www.autoscout24.es/lst/?sort=age&desc=1&bcol=1&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=1&cy=E&body=12&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&"

In [5]:
colours = np.array([1,2,3,4,5,6,7,10,11,12,13,14,15,16])
colours

array([ 1,  2,  3,  4,  5,  6,  7, 10, 11, 12, 13, 14, 15, 16])

In [6]:
colours_dictionary = {'1': 'beige', 
                      '2': 'blue', 
                      '3': 'brown', 
                      '4': 'bronze', 
                      '5': 'yellow', 
                      '6': 'grey', 
                      '7': 'green',
                      '10': 'red',
                      '11': 'black', 
                      '12': 'silver', 
                      '13': 'burgundy', 
                      '14': 'white', 
                      '15': 'orange', 
                      '16': 'gold'}

In [7]:
body =  np.array([1,6,5,12,3,2,4,13,7])
body

array([ 1,  6,  5, 12,  3,  2,  4, 13,  7])

In [8]:
body_type_dictionary = {'1': 'small', 
                    '6': 'sedan', 
                    '5': 'familiar', 
                    '12': 'minivan', 
                    '3': 'coupe', 
                    '2': 'cabrio', 
                    '4': 'suv',
                    '13': 'van',
                    '7' : 'other'} 

In [9]:
# Analysing urls for the diffenret pages:
# url_pag1 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
# url_pag2 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=2&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"
# url_pag3 = "https://www.autoscout24.es/lst/?sort=standard&desc=0&ustate=N%2CU&size=20&page=3&cy=E&atype=C&recommended_sorting_based_id=add4f573-8a91-417b-8217-546c0129def6&"

# Max page per url = 20 (start: page 1, stop: page 21 -1, step: 1 by 1 as the url changes from page=n to page=n+1 from one page to the following

all_pages = np.arange(1,21,1)

In [10]:
all_kms = np.arange(10000,1010000,10000)

In [11]:
#Defining columns for Dealer cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []
body_type = []
seller_type = []

In [ ]:
#Scrapping Dealers

start = time()


for car_colours in colours:
    for car_types in body:
        for all_km in all_kms:
            for all_cars in all_pages:

                all_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&custtype=D&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(all_cars) + "&cy=E&kmto="+ str(all_km) + '&kmfrom='+str(all_km - 9999) + "&body=" + str(car_types) + "&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
                all_cars_soup = BeautifulSoup(all_cars_page.text,"lxml")
                all_cars_ads = all_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

                sleep(randint(2,10))

                for element in all_cars_ads:
                    try:
                        Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
                    except:
                        Title = ""
                    title.append(Title)

                    try:
                        Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
                    except:
                        Version = ""
                    version.append(Version)
                    try:
                        Year = element.find('li', attrs={'data-type':'first-registration'}).text
                    except:
                        Year = ""
                    year.append(Year)
                    try:
                        Kms = element.find('li', attrs={"data-type" : "mileage"}).text
                    except:
                        Kms = ""
                    kms.append(Kms)

                    try:
                        Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
                    except:
                        Gear = ""
                    gear.append(Gear)

                    try:
                        Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
                    except:
                        Fuel = ""
                    fuel.append(Fuel)

                    try:
                        Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
                    except:
                        Fuel_cons = ""
                    fuel_cons.append(Fuel_cons)

                    try:
                        Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
                    except:
                        Co2 = ""
                    co2.append(Co2)

                    try:
                        Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
                    except:
                        Prev_owners = ""
                    prev_owners.append(Prev_owners)
                    try:
                        Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
                    except:
                        Price = ""
                    price.append(Price)

                    try:
                        Hp = element.find('li', text =re.compile(r'CV')).text
                    except:
                        Hp = ""
                    hp.append(Hp)

                    try:
                        Location = element.find('span', attrs={"class":"cldf-summary-seller-contact-zip-city"}).text
                    except:
                        Location = ""
                    location.append(Location)

                    try:
                        Car_colours = car_colours

                    except :
                        Car_colours = ""
                    colour.append(Car_colours)

                    try:
                        Car_types = car_types
                    except:
                        Car_types = ""
                    body_type.append(Car_types)

                        
                        
                        
end = time()

print(end - start)

In [ ]:
#Creating the DataFrame
dealer_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Colour": colour ,
                        "Type" : body_type,
                        "Seller" : "D"})

dealer_cars_df

In [ ]:
#Defining columns for Private cars
title = []
version = []
year = []
kms = []
hp = []
gear = []
fuel = []
fuel_cons = []
co2 =[]
prev_owners = []
price = []
location = []
colour = []
body_type = []
seller_type = []



In [ ]:
#Scrapping Private

start = time()


for car_colours in colours:
    for car_types in body:
        for all_cars in all_pages:

            all_cars_page = requests.get("https://www.autoscout24.es/lst/?sort=age&desc=1&custtype=P&bcol="+ str(car_colours) + "&offer=D%2CJ%2CO%2CS%2CU&ustate=N%2CU&size=20&page=" + str(all_cars) + "&cy=E&kmto="+ str(all_km) + '&kmfrom='+str(all_km - 9999) + "&body=" + str(car_types) + "&atype=C&recommended_sorting_based_id=29e07135-6b02-4c0a-ba49-da018c954028&")
            all_cars_soup = BeautifulSoup(all_cars_page.text,"lxml")
            all_cars_ads = all_cars_soup.find_all('div', attrs={"class": "cl-list-element cl-list-element-gap"})

            sleep(randint(2,10))

            for element in all_cars_ads:
                try:
                    Title = element.find('h2', attrs={"class":'cldt-summary-makemodel sc-font-bold sc-ellipsis'}).text
                except:
                    Title = ""
                title.append(Title)

                try:
                    Version = element.find('h2', attrs={"class":"cldt-summary-version sc-ellipsis"}).text
                except:
                    Version = ""
                version.append(Version)
                try:
                    Year = element.find('li', attrs={'data-type':'first-registration'}).text
                except:
                    Year = ""
                year.append(Year)
                try:
                    Kms = element.find('li', attrs={"data-type" : "mileage"}).text
                except:
                    Kms = ""
                kms.append(Kms)

                try:
                    Gear = element.find('li', attrs={"data-type":"transmission-type"}).text
                except:
                    Gear = ""
                gear.append(Gear)

                try:
                    Fuel = element.find('li', attrs={"class":"summary_item_no_bottom_line"}).text
                except:
                    Fuel = ""
                fuel.append(Fuel)

                try:
                    Fuel_cons = element.find('li', attrs={"data-type":"combined-consumption"}).text
                except:
                    Fuel_cons = ""
                fuel_cons.append(Fuel_cons)

                try:
                    Co2 = element.find('li', attrs={"data-type":"co2-emission"}).text
                except:
                    Co2 = ""
                co2.append(Co2)

                try:
                    Prev_owners = element.find('li', attrs={"data-type" : "previous-owners"}).text
                except:
                    Prev_owners = ""
                prev_owners.append(Prev_owners)
                try:
                    Price = element.find('span', attrs={"class":"cldt-price sc-font-xl sc-font-bold"}).text
                except:
                    Price = ""
                price.append(Price)

                try:
                    Hp = element.find('li', text =re.compile(r'CV')).text
                except:
                    Hp = ""
                hp.append(Hp)

                try:
                    Location = element.find('span', attrs={"class":"cldt-summary-seller-contact-zip-city"}).text
                except:
                    Location = ""
                location.append(Location)

                try:
                    Car_colours = car_colours

                except :
                    Car_colours = ""
                colour.append(Car_colours)

                try:
                    Car_types = car_types
                except:
                    Car_types = ""
                body_type.append(Car_types)

                        
                        
                        
end = time()

print(end - start)

In [ ]:
#Creating the DataFrame
private_cars_df = pd.DataFrame({"Title":title,
                        "Version":version,
                        "Year": year,
                        "Kms": kms,
                        "Hp": hp,
                        "Fuel_type": fuel,
                        "Gear_type": gear,
                        "Fuel_cons": fuel_cons,
                        "CO2_g/Km": co2,
                        "Previous_owners": prev_owners,
                        "City_ZIP" : location,
                        "Price_EUR": price,
                        "Colour": colour ,
                        "Type" : body_type,
                        "Seller" : "P"})
private_cars_df

In [ ]:
#Mergin DataFrames 
all_cars_df = pd.concat([dealer_cars_df,
                         private_cars_df], axis = 0)
#Reset_index
all_cars_df = all_cars_df.reset_index()

In [ ]:


all_cars_df

In [ ]:
all_cars_df.drop(["index"],axis = 1, inplace = True)

In [ ]:
all_cars_df

In [ ]:
all_cars_df.to_csv('/users/alfon/Desktop/Master/TFM/Csv/01.cars_raw_deff.csv', header=True, index=False)